In [9]:
INDEX = ["nasdaq100", "sp500", "nifty500"][0]


In [2]:
import csv
import os, sys
import pickle
import math
from queue import PriorityQueue
from tkinter import N

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import pickle

mpl.rcParams['figure.dpi']= 300

from sklearn.metrics import accuracy_score, ndcg_score
#from torchinfo import summary
from tqdm import tqdm
from utils import (mean_absolute_percentage_error,
                   load_or_create_dataset_graph,
                   mean_square_error, root_mean_square_error)

from models.model_sai import Trans

from torch.nn import Linear, ReLU, Dropout
from torch_geometric.nn import Sequential, GCNConv, JumpingKnowledge
from torch_geometric.nn import global_mean_pool

#import tensorflow_ranking as tfr

# from pytorchltr.loss import LambdaNDCGLoss1, LambdaNDCGLoss2
from torchmetrics.functional import retrieval_normalized_dcg

from random import randint
import wandb
GPU = 3
LR = 0.0006
BS = 128
W = 20
T = 20
LOG = False
D_MODEL = 20
N_HEAD  = 5
DROPOUT = 0.1
D_FF    = 1024
ENC_LAYERS = 1
DEC_LAYERS = 1
MAX_EPOCH = 10
USE_POS_ENCODING = False
USE_GRAPH = [True, False, 'hgat', 'rgat', 'gcn'][2]
HYPER_GRAPH = [True, False][1]
USE_RELATION_GRAPH = ['gcn', 'hypergraph', 'with_sector', False][3]
USE_KG = [True, False][1]
PREDICTION_PROBLEM = 'value'
RUN = randint(1, 100000)
PLOT = False
MODEL_TYPE = '' #'random'
ENCODER_LAYER = ['gru', 'transf', 'lstm'][2]

tau_choices = [5]
tau_positions = [5]

In [7]:

PREDICTION_PROBLEM = 'value'
save_path = "data/pickle/"+INDEX+"/full_graph_data_correct-P25-W"+str(20)+"-T"+str(20)+"_"+str(PREDICTION_PROBLEM)+".pkl"

dataset, company_to_id, graph, hyper_data = load_or_create_dataset_graph(INDEX="nasdaq100", W=20, T=20, save_path=save_path, problem=PREDICTION_PROBLEM, fast=False)


NameError: name 'load_or_create_dataset_graph' is not defined

In [4]:
tau_choices = [1,5,20]
tau_positions = [1, 5, 20]


def collate_fn(instn):
    tkg = instn[0][1]
    instn = instn[0][0]
    print("hellop",file=sys.stderr)
    # df: shape: Companies x W+1 x 5 (5 is the number of features)
    df = torch.Tensor(np.array([x[0] for x in instn])).unsqueeze(dim=2)
    #df = torch.Tensor(np.array([x[1] for x in instn])).unsqueeze(dim=2) - torch.Tensor(np.array([x[2] for x in instn])).unsqueeze(dim=2)
    for i in range(1, 5):
        df1 = torch.Tensor(np.array([x[i] for x in instn])).unsqueeze(dim=2)
        df = torch.cat((df, df1), dim=2)
    min_val = df.min()
    max_val = df.max()

    # Normalize tensor to the range [-1, 1]
    normalized_tensor = 2 * (df - min_val) / (max_val - min_val) - 1
    # Shape: Companies x 1
    target = torch.Tensor(np.array([x[7][tau_pos] for x in instn]))

    # Shape: Companies x 1
    #movement = target >= 1

    best_case, worst_case = torch.Tensor(np.array([x[11][tau_pos+1] for x in instn])), torch.Tensor(np.array([x[10][tau_pos+1] for x in instn]))
    best_case = best_case / torch.Tensor(np.array([x[10][0] for x in instn]))
    worst_case = worst_case / torch.Tensor(np.array([x[11][0] for x in instn]))

    return (normalized_tensor, target, tkg, best_case, worst_case)

In [5]:
# import pickle
# for tau in tau_choices:
#     tau_pos = tau_positions.index(tau)
#     start_time, train_begin = 0, 0
    
#     def collate_fn(instn):
#         tkg = instn[0][1]
#         instn = instn[0][0]
#         print("hellop",file=sys.stderr)
#         # df: shape: Companies x W+1 x 5 (5 is the number of features)
#         df = torch.Tensor(np.array([x[0] for x in instn])).unsqueeze(dim=2)
#         #df = torch.Tensor(np.array([x[1] for x in instn])).unsqueeze(dim=2) - torch.Tensor(np.array([x[2] for x in instn])).unsqueeze(dim=2)
#         for i in range(1, 5):
#             df1 = torch.Tensor(np.array([x[i] for x in instn])).unsqueeze(dim=2)
#             df = torch.cat((df, df1), dim=2)
#         min_val = df.min()
#         max_val = df.max()

#         # Normalize tensor to the range [-1, 1]
#         normalized_tensor = 2 * (df - min_val) / (max_val - min_val) - 1
#         # Shape: Companies x 1
#         target = torch.Tensor(np.array([x[7][tau_pos] for x in instn]))

#         # Shape: Companies x 1
#         #movement = target >= 1

#         best_case, worst_case = torch.Tensor(np.array([x[11][tau_pos+1] for x in instn])), torch.Tensor(np.array([x[10][tau_pos+1] for x in instn]))
#         best_case = best_case / torch.Tensor(np.array([x[10][0] for x in instn]))
#         worst_case = worst_case / torch.Tensor(np.array([x[11][0] for x in instn]))

#         return (normalized_tensor, target, tkg, best_case, worst_case)
#     for phase in range(1, 25):
#         train_loader    = DataLoader(dataset[train_begin:start_time+400], 1, shuffle=True, collate_fn=collate_fn, num_workers=1)
#         val_loader      = DataLoader(dataset[start_time+400:start_time+450], 1, shuffle=False, collate_fn=collate_fn)
#         test_loader     = DataLoader(dataset[start_time+450:start_time+550], 1, shuffle=False, collate_fn=collate_fn)
#         print("Phase: ", phase)
#         with open("pickle/Nasdaq_loaders/train_loader/"+INDEX+str(tau)+"_"+str(phase)+"_25phase.pkl", "wb") as f:
#             pickle.dump(train_loader,f) 
#         with open("pickle/Nasdaq_loaders"+"/val_loader/"+INDEX+str(tau)+"_"+str(phase)+"_25phase.pkl", "wb") as f:
#             pickle.dump(val_loader,f)
#         with open("pickle/Nasdaq_loaders"+"/test_loader/"+INDEX+str(tau)+"_"+str(phase)+"_25phase.pkl", "wb") as f:
#             pickle.dump(test_loader,f)
        
#     #print(len(dataset), len(dataset[start_time:start_time+1000]), len(dataset[start_time+1000:start_time+1100]), len(dataset[start_time+1100:start_time+1400]))
#         start_time += 100
#         if start_time >= 300:
#             train_begin += 100   


In [6]:
start_time, train_begin = 0, 0
train_loader    = DataLoader(dataset[train_begin:start_time+400], 1, shuffle=True, collate_fn=collate_fn, num_workers=1)


In [7]:
import pickle
for tau in tau_choices:
    tau_pos = tau_positions.index(tau)
    start_time, train_begin = 0, 0
    
    def collate_fn(instn):
        tkg = instn[0][1]
        instn = instn[0][0]
        print("hellop",file=sys.stderr)
        # df: shape: Companies x W+1 x 5 (5 is the number of features)
        df = torch.Tensor(np.array([x[0] for x in instn])).unsqueeze(dim=2)
        #df = torch.Tensor(np.array([x[1] for x in instn])).unsqueeze(dim=2) - torch.Tensor(np.array([x[2] for x in instn])).unsqueeze(dim=2)
        for i in range(1, 5):
            df1 = torch.Tensor(np.array([x[i] for x in instn])).unsqueeze(dim=2)
            df = torch.cat((df, df1), dim=2)
        min_val = df.min()
        max_val = df.max()

        # Normalize tensor to the range [-1, 1]
        normalized_tensor = 2 * (df - min_val) / (max_val - min_val) - 1
        # Shape: Companies x 1
        target = torch.Tensor(np.array([x[7][tau_pos] for x in instn]))

        # Shape: Companies x 1
        #movement = target >= 1

        best_case, worst_case = torch.Tensor(np.array([x[11][tau_pos+1] for x in instn])), torch.Tensor(np.array([x[10][tau_pos+1] for x in instn]))
        best_case = best_case / torch.Tensor(np.array([x[10][0] for x in instn]))
        worst_case = worst_case / torch.Tensor(np.array([x[11][0] for x in instn]))

        return (normalized_tensor, target, tkg, best_case, worst_case)
    for phase in range(1, 25):
        train_loader    = DataLoader(dataset[train_begin:start_time+400], 1, shuffle=True, collate_fn=collate_fn, num_workers=1)
        val_loader      = DataLoader(dataset[start_time+400:start_time+450], 1, shuffle=False, collate_fn=collate_fn)
        test_loader     = DataLoader(dataset[start_time+450:start_time+550], 1, shuffle=False, collate_fn=collate_fn)
        print("Phase: ", phase)
        with open("pickle/Nasdaq_loaders/train_loader/"+INDEX+str(tau)+"_"+str(phase)+"_25phase.pkl", "wb") as f:
            pickle.dump(train_loader,f) 
        with open("pickle/Nasdaq_loaders"+"/val_loader/"+INDEX+str(tau)+"_"+str(phase)+"_25phase.pkl", "wb") as f:
            pickle.dump(val_loader,f)
        with open("pickle/Nasdaq_loaders"+"/test_loader/"+INDEX+str(tau)+"_"+str(phase)+"_25phase.pkl", "wb") as f:
            pickle.dump(test_loader,f)
        
    #print(len(dataset), len(dataset[start_time:start_time+1000]), len(dataset[start_time+1000:start_time+1100]), len(dataset[start_time+1100:start_time+1400]))
        start_time += 100
        if start_time >= 300:
            train_begin += 100   

Phase:  1
Phase:  2
Phase:  3
Phase:  4
Phase:  5
Phase:  6
Phase:  7
Phase:  8
Phase:  9
Phase:  10
Phase:  11
Phase:  12
Phase:  13
Phase:  14
Phase:  15
Phase:  16
Phase:  17
Phase:  18
Phase:  19
Phase:  20
Phase:  21
Phase:  22
Phase:  23
Phase:  24
Phase:  1
Phase:  2
Phase:  3
Phase:  4
Phase:  5
Phase:  6
Phase:  7
Phase:  8
Phase:  9
Phase:  10
Phase:  11
Phase:  12
Phase:  13
Phase:  14
Phase:  15
Phase:  16
Phase:  17
Phase:  18
Phase:  19
Phase:  20
Phase:  21
Phase:  22
Phase:  23
Phase:  24
Phase:  1
Phase:  2
Phase:  3
Phase:  4
Phase:  5
Phase:  6
Phase:  7
Phase:  8
Phase:  9
Phase:  10
Phase:  11
Phase:  12
Phase:  13
Phase:  14
Phase:  15
Phase:  16
Phase:  17
Phase:  18
Phase:  19
Phase:  20
Phase:  21
Phase:  22
Phase:  23
Phase:  24


In [5]:
tau_choices = [1,5,20]
tau_positions = [1, 5, 20]
for tau in tau_choices:
    tau_pos = tau_positions.index(tau)
    # print(tau_pos)
    print(tau)

1
5
20


In [12]:
import pickle
with open("pickle/Nasdaq_loaders/train_loader/"+INDEX+str(tau)+"_"+str(1)+"_25phase.pkl", "rb") as f:
    train_loader = pickle.load(f)

AttributeError: Can't get attribute 'collate_fn' on <module '__main__'>